#자연어처리
#이름 : 정주호
#학번 : 2019312072
#전공 : 인공지능융합전공

#수업내용 정리
단어를 파악할 때 그 자체로 이해하기 보다는 주위에 있는 단어들의 의미로 유추한다


word as vector (=embedding) based in counts of neiboring words 
1. Tf-idf 
2. Word2vec 

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
categories = ['rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(newsgroups_train.data)

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)
vectors_test = vectorizer.transform(newsgroups_test.data)
clf = MultinomialNB(alpha=0.01)
clf.fit(vectors, newsgroups_train.target)
pred = clf.predict(vectors_test)
metrics.f1_score(newsgroups_test.target, pred, average='macro')



0.9634773656295091

In [2]:
# remove metadata
newsgroups_test = fetch_20newsgroups(subset='test',remove=('headers', 'footers', 'quotes'), categories=categories)
vectors_test = vectorizer.transform(newsgroups_test.data)

newsgroups_train = fetch_20newsgroups(subset='train',remove=('headers', 'footers', 'quotes'), categories=categories)

vectors = vectorizer.fit_transform(newsgroups_train.data)

clf = MultinomialNB(alpha=0.01)
clf.fit(vectors, newsgroups_train.target)

vectors_test = vectorizer.transform(newsgroups_test.data)
pred = clf.predict(vectors_test)

metrics.f1_score(newsgroups_test.target,pred,average='macro')

0.8512083976713682

#MultinomialNB의 hyperparameter Tuning
GridSearchCV로 하이퍼파라미터 찾기

In [3]:
from sklearn.model_selection import GridSearchCV
parameters = {  
'alpha': (1, 0.1, 0.01, 0.001, 0.0001, 0.00001)  
}  
grid_search= GridSearchCV(clf, parameters)
grid_search.fit(vectors, newsgroups_train.target)

GridSearchCV(cv=None, error_score=nan,
             estimator=MultinomialNB(alpha=0.01, class_prior=None,
                                     fit_prior=True),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': (1, 0.1, 0.01, 0.001, 0.0001, 1e-05)},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [4]:
grid_search.best_estimator_

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [5]:
pred = grid_search.predict(vectors_test)

metrics.f1_score(newsgroups_test.target,pred,average='macro')

0.8723496188818602

#하이퍼파라미터 결과 alpha값이 0.1일때 정확도가 87로 높아짐

#NB와 다른 classification model들을 비교해보기

In [6]:
def model_selection(X_train, y_train, kfold=3):
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.svm import SVC
    from xgboost import XGBClassifier
    from sklearn.model_selection import cross_val_score
    from tqdm.notebook import tqdm
    import pandas as pd

    models = [
        LogisticRegression(),
        XGBClassifier(random_state=100),
        SVC(), DecisionTreeClassifier(max_depth=6),MultinomialNB()
    ]
    
    progress_bar = tqdm(total = len(models) * kfold)
    
    entries = []
    for model in models:
        model_name = model.__class__.__name__
        accuracies = cross_val_score(model, X_train, y_train, 
                        scoring='accuracy', cv=kfold, n_jobs=-1)
    
        for fold_idx, accuracy in enumerate(accuracies):
            progress_bar.update()
            entries.append((model_name, fold_idx, accuracy))
    
    progress_bar.close()
    
    cv_result = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'], index=range(kfold * len(models)))
    return cv_result

In [7]:
model_selection_result = model_selection(vectors, newsgroups_train.target)

  0%|          | 0/15 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
model_selection_result

In [ ]:
 #Logistic Regression hyperparameter tuning 


from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(max_iter=200)
params = {'penalty': ['l2', 'l1'],
       'C': [0.001, 0.01, 0.1, 1, 5, 10]}

grid_lr_clf = GridSearchCV(lr_clf, 
                           param_grid=params, verbose=1, scoring='accuracy', n_jobs=-1, cv = 3)

# eval_metric: 평가 방식. auc = accuracy
# eval_set: 평가 데이터셋
grid_lr_clf.fit(vectors, newsgroups_train.target)

In [ ]:
grid_lr_clf.best_estimator_

In [ ]:
pred = grid_lr_clf.predict(vectors_test)
metrics.f1_score(newsgroups_test.target,pred,average='macro')

In [ ]:
#SVM hyperparameter tuning 


from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
lr_clf = SVC()
params = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}

grid_lr_clf = GridSearchCV(lr_clf, 
                           param_grid=params, verbose=2, scoring='accuracy',cv=2)

# eval_metric: 평가 방식. auc = accuracy
# eval_set: 평가 데이터셋
grid_lr_clf.fit(vectors, newsgroups_train.target)

In [ ]:
grid_lr_clf.best_estimator_

In [ ]:
pred = grid_lr_clf.predict(vectors_test)
metrics.f1_score(newsgroups_test.target,pred,average='macro')

결과 분석 : Logistic Regression, XGBClassifier, SVM, Decision Tree Classifier, MultinomialNB 모델들의 정확도를 비교해본 결과 multinomialNB의 정확도가 가장 높았고, mutinomialNB의 하이퍼파라미터 alpha=0.1로 조정한 결과 87%의 정확도를 낼 수 있었다. 그 뒤로 정확도가 높은 logistic Regression과 SVM의 하이퍼파라미터도 조정해서 정확도를 구해봤는데 84%대로 NB보다 높지 않음

In [ ]:
from sklearn.neural_network import  MLPClassifier
from sklearn.metrics import roc_auc_score

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5,2),random_state=1)

clf.fit(vectors, newsgroups_train.target)

pred = clf.predict(vectors_test)

metrics.f1_score(newsgroups_test.target,pred,average='macro')